<a href="https://colab.research.google.com/github/Junha9/AI/blob/master/Self_Learning_ML_DL/Chapter_05_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
# import statements
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_validate

In [4]:
# downlaod wine data
wine = pd.read_csv('https://bit.ly/wine_csv_data')
data = wine[['alcohol', 'sugar', 'pH']]
target = wine['class']

In [7]:
# split train and test data
train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

In [9]:
# Let's split once again to get the validation set apart from the test set
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)
print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [11]:
# Let's validate the tree model using the validation set. It looks overfitted a lot.
dt = DecisionTreeClassifier(random_state=42)
dt.fit(sub_input, sub_target)
print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))

0.9971133028626413
0.864423076923077


In [14]:
# We can cross validate which uses different validation set during iteration. default is 5-fold cross validation
scores = cross_validate(dt, train_input, train_target)
print(scores)

{'fit_time': array([0.03057265, 0.03849316, 0.01467228, 0.014503  , 0.01507282]), 'score_time': array([0.00821614, 0.00911951, 0.00382733, 0.00360489, 0.00373721]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}


In [17]:
# Let's see the mean value of the test score all the 5 folds
print(np.mean(scores['test_score']))

0.855300214703487
